In [1]:
import os
import pickle
import scipy.io
import numpy as np
from scfw.scopt import scopt
from scfw.frank_wolfe import frank_wolfe
import scfw.portfolio as pr
import matplotlib.pyplot as plt

In [2]:
data_folder = '../data'

problem_name = 'syn_1000_800_10_50'
data = scipy.io.loadmat(os.path.join(data_folder, problem_name + '.mat'))
R = data['W']
results = {problem_name: {}}

In [3]:
np.random.seed(42)

## Data preprocessing and parameters initialization

In [4]:
N, n = R.shape
Mf = 2
nu = 3
sigma_f = 1
diam_X = np.sqrt(2)
rho = np.sqrt(n)

I = np.eye(n)
Iv = np.ones(n)

#running parameters
x0 = np.ones(n) / n
terminate_tol = 1e-20

#parameters for FW
FW_params={
    'iter_FW':50000,
    'line_search_tol':1e-10,
}


sc_params={
    #parameters for SCOPT
    'iter_SC': 1000,
    'Lest': 'backtracking',#,'estimate', #estimate L
    'use_two_phase':False,
    #FISTA parameters
    'fista_type': 'mfista',
    'fista_tol': 1e-5,
    'fista_iter': 1000,
    #Conjugate Gradient Parameters
    'conj_grad_tol':1e-5,
    'conj_grad_iter':1000,
}

## Auxilary functions

In [5]:
func_x = lambda x: pr.portfolio(R,x)
grad_x = lambda x, dot_product: pr.grad_portfolio(R, x, dot_product)
grad_beta = lambda x, s, beta, dot_product, dot_product_s: pr.grad_portfolio(R, (1 - beta) * x + beta * s, (1 - beta) * dot_product + beta * dot_product_s)
hess_x = lambda x, dot_product: pr.hess_portfolio(R, x, dot_product)
hess_mult_x = lambda x, dot_product: pr.hess_mult_portfolio(R, x, dot_product)
hess_mult_vec_x = lambda s, dot_product: pr.hess_mult_vec(R, s, dot_product)
extra_func = lambda x: R @ x
linear_oracle = lambda grad: pr.linear_oracle_simplex(grad)
llo_oracle = lambda x, r, grad: pr.llo_oracle(x, r, grad, rho=rho)
prox_func = lambda x, L: pr.proj_simplex(x)

In [6]:
points = [x0]
 
for _ in range(4):
    x = np.abs(np.random.rand(len(x0)))
    x = x / sum(x)
    points.append(x)

## Run FW

In [7]:
#run_alpha_policies = ["standard","line_search","icml"]
run_alpha_policies = ['lloo']


for policy in run_alpha_policies:
    x, alpha_hist, Gap_hist, Q_hist, time_hist, grad_hist = frank_wolfe(func_x,
                       grad_x,
                       grad_beta,
                       hess_mult_x,
                       extra_func,                                                    
                       Mf,
                       nu,
                       linear_oracle,                                                    
                       x0,
                       FW_params,
                       hess=hess_x,
                       rho=rho,
                       sigma_f=sigma_f,
                       diam_X=diam_X,   
                       lloo_oracle=llo_oracle,                                                 
                       alpha_policy=policy,                                                    
                       eps=terminate_tol, 
                       print_every=10, 
                       debug_info=False)
    
    results[problem_name][policy] = {
        'x': x,
        'alpha_hist': alpha_hist,
        'Gap_hist': Gap_hist,
        'Q_hist': Q_hist,
        'time_hist': time_hist,
        'grad_hist': grad_hist
    }

********* Algorithm starts *********
1265.912766061909
iter = 1, stepsize = 6.358665734316521e-15, criterion = 6.354870570019088e-15, upper_bound=0.05635009446772796, lower_bound=-5.176361103704171


KeyboardInterrupt: 

In [ ]:
run_alpha_policies = ["standard","line_search","icml"]

for policy in run_alpha_policies:
    policy_results = []
    for i in range(len(points)):
        x0 = points[i]
        x, alpha_hist, Gap_hist, Q_hist, time_hist, grad_hist = frank_wolfe(func_x,
                        grad_x,
                        grad_beta,
                        hess_mult_x,
                        extra_func,                                                    
                        Mf,
                        nu,
                        linear_oracle,                                                    
                        x0,
                        FW_params,                                       
                        alpha_policy=policy,                                                    
                        eps=terminate_tol,
                        print_every=100, 
                        debug_info=False)
        
        policy_results.append(
            {
        'Q_hist': Q_hist,
        'time_hist': time_hist,}
        )
    with open('./results/portfolio/' + problem_name + '_' + policy + '.pckl', 'wb') as f:
        pickle.dump(policy_results, f)
#    results[problem_name][policy] = policy_results

## Run SCOPT

In [ ]:
scopt_results = []
for i in range(len(points)):
    x0 = points[i]
    x, alpha_hist, Q_hist, time_hist, grad_hist = scopt(func_x,
            grad_x,
            hess_mult_x,
            hess_mult_vec_x,
            Mf,
            nu,
            prox_func,
            x0,  
            sc_params,                                              
            eps=terminate_tol,                                              
            print_every=1)
        
    scopt_results.append({
        'Q_hist': Q_hist,
        'time_hist': time_hist,
    })

iter =    1, stepsize = 5.365e-01, rdiff = 5.446e-01 , f = 0.0563501

Fista err = 9.972e-06; Subiter = 304; subproblem converged!

iter =    2, stepsize = 7.169e-01, rdiff = 2.508e-01 , f = -2.51766

Fista err = 9.913e-06; Subiter =  88; subproblem converged!

iter =    3, stepsize = 8.994e-01, rdiff = 7.167e-02 , f = -4.00572

Fista err = 9.893e-06; Subiter =  17; subproblem converged!

iter =    4, stepsize = 9.889e-01, rdiff = 7.217e-03 , f = -4.51508

Fista err = 1.703e-11; Subiter =   2; subproblem converged!

iter =    5, stepsize = 9.999e-01, rdiff = 9.456e-05 , f = -4.57079

Fista err = 7.056e-07; Subiter =   2; subproblem converged!

iter =    6, stepsize = 1.000e+00, rdiff = 2.784e-05 , f = -4.57141

Fista err = 4.533e-06; Subiter =   2; subproblem converged!

iter =    7, stepsize = 1.000e+00, rdiff = 1.541e-05 , f = -4.57141

Fista err = 1.220e-10; Subiter =   2; subproblem converged!

boo
Convergence achieved!
iter =    8, stepsize = 0.000e+00, rdiff = 0.000e+00,value=-4.5